In [1]:
from pathlib import Path

dataset_path = Path("./Dataset/Aircraft_Fuselage_DET2023/aircraft_fuselage_yolo")
labels = sorted(dataset_path.rglob("*labels_full/*.txt"))[:-1]  # all data in 'labels'
labels[-1]


PosixPath('Dataset/Aircraft_Fuselage_DET2023/aircraft_fuselage_yolo/labels_full/414.txt')

In [2]:
import yaml

yaml_file = "AFDET_dataset.yaml"  # your data YAML with data directories and names dictionary
with open(yaml_file, encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())
cls_idx

[0, 1, 2, 3]

In [3]:
import pandas as pd

index = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=index)

In [4]:
from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label) as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ", 1)[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

In [5]:
labels_df

,0,1,2,3
001,2.0,0.0,0.0,0.0
002,5.0,0.0,0.0,0.0
003,2.0,0.0,0.0,0.0
004,6.0,0.0,0.0,0.0
005,0.0,1.0,0.0,0.0
...,...,...,...,...
410,0.0,0.0,0.0,1.0
411,1.0,0.0,0.0,0.0
412,1.0,0.0,0.0,0.0
413,1.0,0.0,0.0,0.0


In [6]:
import random

from sklearn.model_selection import KFold

random.seed(0)  # for reproducibility
ksplit = 6
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [7]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=index, columns=folds)

for i, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{i}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{i}"].loc[labels_df.iloc[val].index] = "val"

In [8]:
folds

['split_1', 'split_2', 'split_3', 'split_4', 'split_5', 'split_6']

In [9]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [10]:
fold_lbl_distrb

,0,1,2,3
split_1,0.243802,0.139344,0.147368,0.125
split_2,0.248963,0.139344,0.172043,0.285714
split_3,0.223577,0.198276,0.147368,0.246154
split_4,0.148855,0.287037,0.345679,0.140845
split_5,0.162162,0.177966,0.329268,0.306452
split_6,0.180392,0.275229,0.10101,0.125


In [11]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images_full").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

In [12]:
import shutil

from tqdm import tqdm

for image, label in tqdm(zip(images, labels), total=len(images), desc="Copying files"):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

Copying files: 100%|██████████| 389/389 [00:00<00:00, 2787.14it/s]


In [13]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

In [14]:
from ultralytics import RTDETR
import albumentations as A


In [15]:
results = {}

# Define your additional arguments here
batch = 8
project = "kfold_demo"
epochs = 40

for k, dataset_yaml in enumerate(ds_yamls):
    A.MotionBlur(p=0.2),
    A.Rotate(limit=20, p=0.3),
    model = RTDETR("rtdetr-l.pt")
    results[k] = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch,
        project=project,
        name=f"fold_{k + 1}",
        imgsz=640,
        patience=5,
    )

New https://pypi.org/project/ultralytics/8.4.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.7 🚀 Python-3.11.9 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12029MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Dataset/Aircraft_Fuselage_DET2023/aircraft_fuselage_yolo/2026-01-29_6-Fold_Cross-val/split_1/split_1_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=r

/home/kaizen/anaconda3/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647429097/work/aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/40      6.61G      1.852      36.37      1.337          5        640: 100% ━━━━━━━━━━━━ 41/41 2.0it/s 20.4s0.4ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 5.8it/s 0.9s0.3s
                   all         65         99        0.5     0.0556    0.00103   0.000384

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/home/kaizen/anaconda3/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647429097/work/aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/40       6.8G      1.658      26.93     0.8675          8        640: 100% ━━━━━━━━━━━━ 41/41 2.1it/s 19.5s0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 6.2it/s 0.8s0.3s
                   all         65         99    0.00116     0.0833    0.00196   0.000844

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/40      6.85G     0.9266      20.88     0.4088         17        640: 0% ──────────── 0/41  0.4s

...

      36/40      6.75G     0.4513     0.6465     0.1135          9        640: 100% ━━━━━━━━━━━━ 41/41 2.4it/s 17.2s0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 5.1it/s 0.8s0.3s
                   all         64         95      0.494      0.591      0.516      0.291

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/home/kaizen/anaconda3/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647429097/work/aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      37/40      6.85G     0.4581      0.626     0.1145          7        640: 100% ━━━━━━━━━━━━ 41/41 2.4it/s 17.4s0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 5.1it/s 0.8s0.3s
                   all         64         95      0.526      0.625      0.537      0.302

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      38/40       6.8G       0.46     0.5607     0.1236         18        640: 0% ──────────── 0/41  0.4s

/home/kaizen/anaconda3/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647429097/work/aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      38/40       6.8G     0.4527     0.6485      0.123          5        640: 100% ━━━━━━━━━━━━ 41/41 2.4it/s 17.2s0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 5.1it/s 0.8s0.3s
                   all         64         95      0.523      0.624      0.544      0.304
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 33, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

38 epochs completed in 0.238 hours.
Optimizer stripped from /media/kaizen/T7/Project/final_project/Project/runs/detect/kfold_demo/fold_6/weights/last.pt, 66.2MB
Optimizer stripped from /media/kaizen/T7/Project/final_project/Project/runs/detect/kfold_demo/fold_6/weights/best.pt, 66.2MB

Validating /media/kaizen/T7/Project/final_project/Project/runs/detect/kfold_demo/fold_6/weights/b